In [1]:
import os

import pickle
import pandas as pd

In [2]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [3]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [4]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [5]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [6]:
print(f'Answer to Q1 is: {round(y_pred.std(), 3)}')

Answer to Q1 is: 6.247


In [7]:
df['ride_id'] = f'{2023:04d}/{3:02d}_' + df.index.astype('str')
df['predicted_duration'] = y_pred
df_result = df[['ride_id', 'predicted_duration']]
df_result.to_parquet(
    'predictions.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)


In [8]:
file_size = os.path.getsize('predictions.parquet')
print(f'Answer to Q2 is: {file_size / (1024 * 1024):.2f} M')

Answer to Q2 is: 65.46 M


In [9]:
print(f'Answer to Q3 is: nbconvert')

Answer to Q3 is: nbconvert
